## 使用DeepSeek进行数据表的查询

In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor

db_user = "student123"
db_password = "student321"
#db_host = "localhost:3306"
db_host = "rm-uf6z891lon6dxuqblqo.mysql.rds.aliyuncs.com:3306"
db_name = "action"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
db

In [3]:
from langchain.chat_models import ChatOpenAI
import os

# 从环境变量获取 dashscope 的 API Key
api_key = os.environ.get('DASHSCOPE_API_KEY')

# 通过LLM => 撰写SQL
llm = ChatOpenAI(
    temperature=0.01,
    model="deepseek-v3",  
    #model = "qwen-turbo",
    # openai_api_key = "sk-9846f14a2104490b960adbf5c5b3b32e",
    # openai_api_base="https://api.deepseek.com"
    openai_api_base = "https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key  = api_key
)

# 需要设置llm
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# SQL智能体：给它目标，它自己会进行规划，最终把结果给你
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

C:\Users\cheny\AppData\Local\Temp\ipykernel_71516\3561425379.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
# Task: 描述数据表
agent_executor.run("描述与订单相关的表及其关系")

C:\Users\cheny\AppData\Local\Temp\ipykernel_71516\2182499378.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("描述与订单相关的表及其关系")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI see several tables that might be related to orders, particularly "orders" and "customers". I should examine their schemas to understand their structure and relationships.

Action: sql_db_schema
Action Input: "orders, customers"
CREATE TABLE customers (
	`Id` INTEGER, 
	`Name` VARCHAR(255)
)DEFAULT CHARSET=utf8mb3 ENGINE=InnoDB

/*
3 rows from customers table:
Id	Name
1	Joe
2	Henry
3	Sam
*/


CREATE TABLE orders (
	`Id` INTEGER, 
	`CustomerId` INTEGER
)DEFAULT CHARSET=utf8mb3 ENGINE=InnoDB

/*
3 rows from orders table:
Id	CustomerId
1	3
2	1
*/

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `From examining the schema, I can see the following about the order-related tables and their relationships:

1. Tables involved:
- `customers` table: Stores customer information with columns `Id` (primary key) and `Name`
- `orders` table: Stores order information with columns `Id` (primary key) and `CustomerId` (foreign key)

2. Relationship:
- The `orders` table has a foreign key relationship with the `customers` table through the `CustomerId` column
- This is a one-to-many relationship where one customer can have many orders
- The sample data shows:
  - Customer "Sam" (Id=3) has order Id=1
  - Customer "Joe" (Id=1) has order Id=2
  - Customer "Henry" (Id=2) has no orders in the sample data

This appears to be a simple order tracking system where orders are linked to the

In [7]:
# 这个任务，实际上数据库中 没有HeroDetails表
agent_executor.run("描述HeroDetails表")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: 
address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI see that the "heros" table is listed, which seems relevant to the question about "HeroDetails." I will now query the schema of the "heros" table to understand its structure.

Action: sql_db_schema
Action Input: heros


CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, 
	hp_max FLOAT, 
	hp_growth FLOAT, 
	hp_start FLOAT, 
	mp_max FLOAT, 
	mp_growth FLOAT, 
	mp_start FLOAT, 
	attack_max FLOAT, 
	attack_growth FLOAT, 
	attack_start FLOAT, 
	defense_max FLOAT, 
	defense_growth FLOAT, 
	defense_start FLOAT, 
	hp_5s_max FLOAT, 
	hp_5s_growth FLOAT, 
	hp_5s_start FLOAT, 
	mp_5s_max FLOAT, 
	mp_5s_growth FLOAT, 
	

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `The "heros" table contains detailed information about various heroes, including their attributes such as health points (HP), mana points (MP), attack, defense, and other growth-related statistics. The table also includes information about the hero's main role, assist role, and birthdate. Here is a summary of the columns in the "heros" table:

- **id**: Unique identifier for each hero (Integer, Primary Key).
- **name**: Name of the hero (Varchar, Not Null).
- **hp_max**: Maximum health points (Float).
- **hp_growth**: Health points growth rate (Float).
- **hp_start**: Starting health points (Float).
- **mp_max**: Maximum mana points (Float).
- **mp_growth**: Mana points growth rate (Float).
- **mp_start**: Starting mana points (Float).
- **attack_max**: Maximum attack pow

In [3]:
agent_executor.run("描述Hero表")

C:\Users\cheny\AppData\Local\Temp\ipykernel_60860\828330074.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("描述Hero表")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherI now know the tables available in the database. Next, I need to determine which tables are relevant to the question about the Hero table. Since the question specifically asks about the Hero table, I will query its schema.
Action: sql_db_schema
Action Input: heros
CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL, 
	hp_max FLOAT, 
	hp_growth FLOAT, 
	hp_start FLOAT, 
	mp_max FLOAT, 
	mp_growth FLOAT, 
	mp_start FLOAT, 
	attack_max FLOAT, 
	attack_growth FLOAT, 
	attack_start FLOAT, 
	defense_max FLOAT, 
	defense_growth FLOAT, 
	defense_start FLOAT, 
	hp_5s_max FLOAT, 
	hp_5s_growth FLOAT, 
	hp_5s_start FLOAT, 

'The Hero table contains the following columns:\n\n- id: Integer, Auto-incremented primary key\n- name: String (up to 255 characters), representing the name of the hero\n- hp_max: Float, maximum health points\n- hp_growth: Float, health points growth rate\n- hp_start: Float, starting health points\n- mp_max: Float, maximum mana points\n- mp_growth: Float, mana points growth rate\n- mp_start: Float, starting mana points\n- attack_max: Float, maximum attack power\n- attack_growth: Float, attack power growth rate\n- attack_start: Float, starting attack power\n- defense_max: Float, maximum defense power\n- defense_growth: Float, defense power growth rate\n- defense_start: Float, starting defense power\n- hp_5s_max: Float, maximum health regeneration per 5 seconds\n- hp_5s_growth: Float, health regeneration growth rate per 5 seconds\n- hp_5s_start: Float, starting health regeneration per 5 seconds\n- mp_5s_max: Float, maximum mana regeneration per 5 seconds\n- mp_5s_growth: Float, mana rege

In [7]:
agent_executor.run("找出英雄攻击力最高的前5个英雄")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: 
address, asset_grades, bank, car_sales, customers, dept, employee, form, height_grades, hero_score, heros, orders, person, player, player_score, student, team, team_score, test_work, trips, user, users, using, weatherThe most relevant table for this question seems to be "heros" or "hero_score". I will check the schema of these tables to find the relevant columns for querying the heroes with the highest attack power.

Action: sql_db_schema
Action Input: heros, hero_score


CREATE TABLE hero_score (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	score INTEGER NOT NULL DEFAULT '0', 
	name VARCHAR(20) CHARACTER SET utf8mb4 COLLATE utf8mb4_0900_ai_ci, 
	PRIMARY KEY (id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from hero_score table:
id	score	name
1	80	张飞
2	95	关羽
3	92	刘备
*/


CREATE TABLE heros (
	id INTEGER NOT NULL AUTO_INCREMENT, 
	name VARCHAR(255) CHARACTER SET utf8 COLLATE

'攻击力最高的前5个英雄是：阿轲 (427.0), 孙尚香 (411.0), 百里守约 (410.0), 虞姬 (407.0), 黄忠 (403.0)。'